# Задача 18
## Распознавание британских гласных

Распознавание британских гласных (11 штук) по данным с динамиков, рекомендуется использовать нормированные признаки (файл .scaled). Решить задачу многоклассовой классификации с помощью решающего дерева. Реализовать метод решающего дерева, построить область разделения на классы в проекции на любые 2 признака.

# Д/З № 2

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Read data
Данные https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass.html

In [2]:
data = pd.read_csv('../../data/vowel.scale', sep = ' ', header=None)

# clean data
for i in range(1,11):
    data[i] = data[i].apply(lambda x: x.split(':')[1]).astype('float64')
    
data.rename({0:'y'}, axis=1, inplace=True)

data.drop(11, axis=1, inplace=True)

In [3]:
data.sample()

,y,1,2,3,4,5,6,7,8,9,10
44,0,-0.242155,-0.507246,-0.125641,0.731111,-0.013643,0.579513,-0.047619,0.204316,-0.232717,-0.509753


In [4]:
data, test = train_test_split(data, shuffle=True, stratify=data['y'], test_size=0.2,  random_state=11)

In [5]:
tr_col = list(range(1,11))
X = data[tr_col].values
y = data['y'].values

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
model = DecisionTreeClassifier(random_state=4)
score = cross_val_score(model, X, y, cv=cv, scoring='accuracy')

score.mean(), score.std()

(0.8285831285831285, 0.04203126246750487)

In [ ]:
parameters={
    'min_samples_split' : range(2,20,2),
    'max_depth': range(1,20,2),
    'max_features': range(1,11)    
}
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
model = DecisionTreeClassifier(random_state=4)

clf = GridSearchCV(model, parameters, scoring='accuracy', n_jobs=-1, cv=cv)
clf.fit(X,y)

In [ ]:
best_model = clf.best_estimator_

score = cross_val_score(best_model, X, y, cv=cv, scoring='accuracy')

score.mean(), score.std()

### Проверяем на отложенной выборке

In [ ]:
pred = best_model.predict(test[tr_col])
accuracy_score(test['y'], pred)

In [ ]:
def plot_forest(X, y, model):
    colors = ['black', 'grey', 'brown', 'orange', 'olive', 'green',
         'cyan', 'blue', 'purple', 'red', 'white']
    
    plt.figure(figsize=(20,10))
    ax = plt.gca()
    h = 0.02

    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    model.fit(X, y)
    
    Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=.4)
    ax.set_title("Разделяющая поверхность" , size=24)

    ax.scatter(X[:, 0], X[:, 1], c=np.array(colors)[y], s=60)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xticks(())
    ax.set_yticks(())

In [ ]:
plot_forest(X[:,:2], y, model)

# Д/З № 3

### Построить график зависимости ошибки и стандартного отклонения от объема выборки.

In [ ]:
def mean_std(n):
    score = cross_val_score(best_model, X[:n], y[:n], scoring='accuracy', cv=2)
    return {'mean':score.mean(), 'std':score.std()}

In [ ]:
mean = []
std = []
for n in range(40, len(X), 1):
    res = mean_std(n)
    mean.append(res['mean'])
    std.append(res['std'])

In [ ]:
plt.figure(figsize=(16,12))
plt.grid(True)

plt.errorbar(range(40, len(X), 1),mean, yerr=std) 
plt.xlabel('Count object', size=24)
plt.ylabel('Accuracy', size=24)

plt.savefig(fname='std.png',format='png')
plt.show()